In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from neural_net import feedForwardNet
from pendulum_utils import terminalPendulum, Solver, Datagen
import matplotlib.pyplot as plt
import example_robot_data
import crocoddyl
from crocoddyl.utils.pendulum import CostModelDoublePendulum, ActuationModelDoublePendulum

In [44]:
neural_net = torch.load('./nets/net2.pth')

In [51]:
class terminalPendulum(crocoddyl.ActionModelAbstract):
    """
    This includes a feedforward network in crocoddyl
    
    """
    def __init__(self,neural_net, robot):
        crocoddyl.ActionModelAbstract.__init__(self,crocoddyl.StateMultibody(robot),0,0)
        self.net = neural_net
        self.net.double()
    def calc(self, data, x, u=None):
        print(type(x[1]))
        state = torch.from_numpy(x)
        #print(state, "\n")
        # Get the cost
        with torch.no_grad():
            data.cost = self.net(state).item()
        print(data.cost)    

    def calcDiff(self, data, x, u=None):
        pass
            


In [52]:
robot = example_robot_data.loadDoublePendulum()
robot_model = robot.model
terminal_model = terminalPendulum(neural_net, robot_model)

state = crocoddyl.StateMultibody(robot_model)
actModel = ActuationModelDoublePendulum(state, actLink=1)

weights = np.array([1.5, 1.5, 1, 1] + [0.1] * 2)
runningCostModel = crocoddyl.CostModelSum(state, actModel.nu)
dt = 1e-2

xRegCost = crocoddyl.CostModelState(state, 
                                    crocoddyl.ActivationModelQuad(state.ndx),
                                    state.zero(),
                                    actModel.nu)

uRegCost = crocoddyl.CostModelControl(state,
                                      crocoddyl.ActivationModelQuad(1),
                                      actModel.nu)
xPendCost = CostModelDoublePendulum(state, 
                                    crocoddyl.ActivationModelWeightedQuad(weights),
                                    actModel.nu)

runningCostModel.addCost("uReg", uRegCost, 1e-4 / dt)
runningCostModel.addCost("xGoal", xPendCost, 1e-5 / dt)

runningModel = crocoddyl.IntegratedActionModelEuler(
                                                crocoddyl.DifferentialActionModelFreeFwdDynamics(state,
                                                actModel,
                                                runningCostModel),
                                                dt)
x0 = np.array([-3.144, 3.14, 0.4, 0.4])
problem = crocoddyl.ShootingProblem(x0, [runningModel] * 100, terminal_model)
#            if use_fddp:
fddp = crocoddyl.SolverFDDP(problem)
fddp.solve([], [], 1000)

<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float64'>
nan
<class 'numpy.float6

False

In [34]:
torch.DoubleTensor

torch.DoubleTensor

In [50]:
neural_net.double()

feedForwardNet(
  (activation): Tanh()
  (fc1): Linear(in_features=4, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=1, bias=True)
)